In [6]:
import pandas as pd
import re
from sqlalchemy import create_engine

In [7]:
# Charger le dataset
games = pd.read_csv('gamesIGDB.csv')  # Remplacez par le nom de votre fichier
games

,id,category,cover,first_release_date,name,platforms,rating,release_dates,slug,summary,genres
0,269473,0,334181.0,1.696550e+09,Resonite,"[3, 6, 163]",100.000000,"[522725, 522726, 522727]",resonite--1,Enter a novel digital universe with infinite p...,NaN
1,117294,0,398099.0,1.701907e+09,Throne and Liberty,"[6, 167, 169]",100.000000,"[556711, 620030, 620031, 620033, 641310, 64131...",throne-and-liberty,Throne and Liberty is a free-to-play MMORPG th...,"[12, 31]"
2,264773,0,337256.0,1.695773e+09,Lizards Must Die,[6],99.798620,[517965],lizards-must-die,The great exploits of the Ancient Rus in the w...,"[25, 32]"
3,112659,3,69819.0,1.543277e+09,Batman: Arkham Collection,"[6, 48, 49]",99.727196,"[209453, 209454, 209455, 209456]",batman-arkham-collection,"""Batman: Arkham Collection brings you the defi...","[12, 25, 31]"
4,173172,3,184733.0,1.632787e+09,Outer Wilds: Archaeologist Edition,"[6, 48, 49, 130, 167, 169]",99.655340,"[317893, 317894, 317895, 536374, 536745, 53674...",outer-wilds-archaeologist-edition,Outer Wilds: Archaeologist Edition contains Ou...,"[9, 31, 32]"
...,...,...,...,...,...,...,...,...,...,...,...
291527,110,0,118268.0,7.258464e+08,Freddy Pharkas: Frontier Pharmacist,"[6, 13, 14]",NaN,"[298, 299, 300]",freddy-pharkas-frontier-pharmacist,"The adventures of Freddy Pharkas, a pharmacist...","[2, 31]"
291528,108,0,109.0,9.466848e+08,Hidden & Dangerous: Devil's Bridge,[6],NaN,[497],hidden-dangerous-devil-s-bridge,NaN,"[5, 15, 24]"
291529,88,0,98121.0,1.008288e+09,Frank Herbert's Dune,"[6, 8]",NaN,"[31040, 31041, 31042]",frank-herbert-s-dune,"In the year 10191, the Houses Atreides and Har...","[9, 31]"
291530,48,0,105006.0,1.245715e+09,Overlord: Minions,[20],NaN,[792],overlord-minions,"Meet Giblet, Blaze, Stench, and Zap - an elite...","[9, 25]"


In [10]:
covers = pd.read_csv("gamecoversIGDB.csv")
covers = covers.rename(columns={'id': 'cover'})
covers

/var/folders/60/lfs1wmrj5_x9wglj8m089x300000gn/T/ipykernel_17521/966374923.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  covers = pd.read_csv("gamecoversIGDB.csv")


,cover,game,height,image_id,url,width,checksum,alpha_channel,animated,game_localization
0,65483,100547.0,800.0,pkkd08qhfxt5cosp3hra,//images.igdb.com/igdb/image/upload/t_thumb/pk...,1260.0,e09f916b-04fe-8993-594e-b18435438a35,NaN,NaN,NaN
1,243407,216446.0,900.0,co57tb,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,eb1b2bda-88c5-8a0d-bfce-730000505844,False,False,NaN
2,334179,269472.0,1600.0,co75ur,//images.igdb.com/igdb/image/upload/t_thumb/co...,1200.0,1e5c0437-a711-e9b4-5377-37b9f9da6cb8,False,False,NaN
3,136720,145578.0,800.0,co2xhs,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,f369e510-d1a2-ddf5-fdc2-24fef827ccd0,False,False,NaN
4,334194,13274.0,1400.0,co75v6,//images.igdb.com/igdb/image/upload/t_thumb/co...,1050.0,0b3df5b0-764e-3dfd-5a70-a82f399179f6,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...
229428,396292,308198.0,400.0,co8hs4,//images.igdb.com/igdb/image/upload/t_thumb/co...,640.0,21ea735e-5cef-274f-6bb2-102abb8e5266,False,False,NaN
229429,396301,275562.0,900.0,co8hsd,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,d03578ef-477f-07db-e757-21800baf2a6a,True,False,NaN
229430,400159,145931.0,800.0,co8krj,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,1521d25f-88a8-ed05-bebf-ad2d5e5dbe18,True,False,NaN
229431,396312,43220.0,800.0,co8hso,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,d65ad3aa-a877-5c81-477e-517ec286e34a,False,False,NaN


In [12]:
test = pd.merge(games, covers, how='left', on='cover')
test

,id,category,cover,first_release_date,name,platforms,rating,release_dates,slug,summary,genres,game,height,image_id,url,width,checksum,alpha_channel,animated,game_localization
0,269473,0,334181.0,1.696550e+09,Resonite,"[3, 6, 163]",100.000000,"[522725, 522726, 522727]",resonite--1,Enter a novel digital universe with infinite p...,NaN,269473.0,800.0,co75ut,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,4ae2c815-834c-732e-e211-d3cf7c30bb28,False,False,NaN
1,117294,0,398099.0,1.701907e+09,Throne and Liberty,"[6, 167, 169]",100.000000,"[556711, 620030, 620031, 620033, 641310, 64131...",throne-and-liberty,Throne and Liberty is a free-to-play MMORPG th...,"[12, 31]",117294.0,800.0,co8j6b,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,79fd5e31-bc11-603c-21a9-f8b72eab2e5e,False,False,NaN
2,264773,0,337256.0,1.695773e+09,Lizards Must Die,[6],99.798620,[517965],lizards-must-die,The great exploits of the Ancient Rus in the w...,"[25, 32]",264773.0,800.0,co7888,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,71f559ec-d8c6-d6e7-1fc6-cff5d8d9468b,False,False,NaN
3,112659,3,69819.0,1.543277e+09,Batman: Arkham Collection,"[6, 48, 49]",99.727196,"[209453, 209454, 209455, 209456]",batman-arkham-collection,"""Batman: Arkham Collection brings you the defi...","[12, 25, 31]",112659.0,720.0,co1hvf,//images.igdb.com/igdb/image/upload/t_thumb/co...,540.0,6b879be4-108a-453b-6a15-b69f6389f38e,True,False,NaN
4,173172,3,184733.0,1.632787e+09,Outer Wilds: Archaeologist Edition,"[6, 48, 49, 130, 167, 169]",99.655340,"[317893, 317894, 317895, 536374, 536745, 53674...",outer-wilds-archaeologist-edition,Outer Wilds: Archaeologist Edition contains Ou...,"[9, 31, 32]",173172.0,1024.0,co3yjh,//images.igdb.com/igdb/image/upload/t_thumb/co...,768.0,19fe136d-0d99-737b-5f7e-97c84151db5e,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291527,110,0,118268.0,7.258464e+08,Freddy Pharkas: Frontier Pharmacist,"[6, 13, 14]",NaN,"[298, 299, 300]",freddy-pharkas-frontier-pharmacist,"The adventures of Freddy Pharkas, a pharmacist...","[2, 31]",110.0,858.0,co2j98,//images.igdb.com/igdb/image/upload/t_thumb/co...,643.0,e2f38d84-0d54-5567-424b-5280ff9ced6c,False,False,NaN
291528,108,0,109.0,9.466848e+08,Hidden & Dangerous: Devil's Bridge,[6],NaN,[497],hidden-dangerous-devil-s-bridge,NaN,"[5, 15, 24]",108.0,500.0,pmjszoa4lgd7nznewajq,//images.igdb.com/igdb/image/upload/t_thumb/pm...,384.0,e057cbf1-a627-8cb4-6fc5-021a5e06403d,NaN,NaN,NaN
291529,88,0,98121.0,1.008288e+09,Frank Herbert's Dune,"[6, 8]",NaN,"[31040, 31041, 31042]",frank-herbert-s-dune,"In the year 10191, the Houses Atreides and Har...","[9, 31]",88.0,1067.0,co23pl,//images.igdb.com/igdb/image/upload/t_thumb/co...,800.0,eb16205c-8c3f-b75c-41d3-0bb3f75d0a59,False,False,NaN
291530,48,0,105006.0,1.245715e+09,Overlord: Minions,[20],NaN,[792],overlord-minions,"Meet Giblet, Blaze, Stench, and Zap - an elite...","[9, 25]",48.0,952.0,co290u,//images.igdb.com/igdb/image/upload/t_thumb/co...,714.0,ebe98ce6-1102-44e2-f90b-bad6df729deb,False,False,NaN


In [13]:
test.to_csv('test.csv', index=False)

In [14]:
df = pd.read_csv("cleantest.csv")
df

,id,category,cover,first_release_date,name,platforms,rating,release_dates,slug,summary,...,game,height,image_id,url,width,checksum,alpha_channel,animated,game_localization,summary_clean
0,269473,0,334181.0,1.696550e+09,Resonite,"[3, 6, 163]",100.000000,"[522725, 522726, 522727]",resonite--1,Enter a novel digital universe with infinite p...,...,269473.0,800.0,co75ut,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,4ae2c815-834c-732e-e211-d3cf7c30bb28,False,False,NaN,enter a novel digital universe with infinite p...
1,117294,0,398099.0,1.701907e+09,Throne and Liberty,"[6, 167, 169]",100.000000,"[556711, 620030, 620031, 620033, 641310, 64131...",throne-and-liberty,Throne and Liberty is a free-to-play MMORPG th...,...,117294.0,800.0,co8j6b,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,79fd5e31-bc11-603c-21a9-f8b72eab2e5e,False,False,NaN,throne and liberty is a freetoplay mmorpg that...
2,264773,0,337256.0,1.695773e+09,Lizards Must Die,[6],99.798620,[517965],lizards-must-die,The great exploits of the Ancient Rus in the w...,...,264773.0,800.0,co7888,//images.igdb.com/igdb/image/upload/t_thumb/co...,600.0,71f559ec-d8c6-d6e7-1fc6-cff5d8d9468b,False,False,NaN,the great exploits of the ancient rus in the w...
3,112659,3,69819.0,1.543277e+09,Batman: Arkham Collection,"[6, 48, 49]",99.727196,"[209453, 209454, 209455, 209456]",batman-arkham-collection,"""Batman: Arkham Collection brings you the defi...",...,112659.0,720.0,co1hvf,//images.igdb.com/igdb/image/upload/t_thumb/co...,540.0,6b879be4-108a-453b-6a15-b69f6389f38e,True,False,NaN,batman arkham collection brings you the defini...
4,173172,3,184733.0,1.632787e+09,Outer Wilds: Archaeologist Edition,"[6, 48, 49, 130, 167, 169]",99.655340,"[317893, 317894, 317895, 536374, 536745, 53674...",outer-wilds-archaeologist-edition,Outer Wilds: Archaeologist Edition contains Ou...,...,173172.0,1024.0,co3yjh,//images.igdb.com/igdb/image/upload/t_thumb/co...,768.0,19fe136d-0d99-737b-5f7e-97c84151db5e,True,False,NaN,outer wilds archaeologist edition contains out...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249913,146,2,274864.0,1.115251e+09,Star Wars Galaxies: Rage of the Wookiees,[6],NaN,[124710],star-wars-galaxies-rage-of-the-wookiees,Star Wars Galaxies: Rage of the Wookiees was t...,...,146.0,1011.0,co5w34,//images.igdb.com/igdb/image/upload/t_thumb/co...,758.0,02068892-47ab-ad6f-7275-a2e5f03a644f,False,False,NaN,star wars galaxies rage of the wookiees was th...
249914,110,0,118268.0,7.258464e+08,Freddy Pharkas: Frontier Pharmacist,"[6, 13, 14]",NaN,"[298, 299, 300]",freddy-pharkas-frontier-pharmacist,"The adventures of Freddy Pharkas, a pharmacist...",...,110.0,858.0,co2j98,//images.igdb.com/igdb/image/upload/t_thumb/co...,643.0,e2f38d84-0d54-5567-424b-5280ff9ced6c,False,False,NaN,the adventures of freddy pharkas a pharmacist ...
249915,88,0,98121.0,1.008288e+09,Frank Herbert's Dune,"[6, 8]",NaN,"[31040, 31041, 31042]",frank-herbert-s-dune,"In the year 10191, the Houses Atreides and Har...",...,88.0,1067.0,co23pl,//images.igdb.com/igdb/image/upload/t_thumb/co...,800.0,eb16205c-8c3f-b75c-41d3-0bb3f75d0a59,False,False,NaN,in the year the houses atreides and harkonnen ...
249916,48,0,105006.0,1.245715e+09,Overlord: Minions,[20],NaN,[792],overlord-minions,"Meet Giblet, Blaze, Stench, and Zap - an elite...",...,48.0,952.0,co290u,//images.igdb.com/igdb/image/upload/t_thumb/co...,714.0,ebe98ce6-1102-44e2-f90b-bad6df729deb,False,False,NaN,meet giblet blaze stench and zap an elite mini...


In [15]:
df["url"] = "https:" + df["url"]
df["url"]

0         https://images.igdb.com/igdb/image/upload/t_th...
1         https://images.igdb.com/igdb/image/upload/t_th...
2         https://images.igdb.com/igdb/image/upload/t_th...
3         https://images.igdb.com/igdb/image/upload/t_th...
4         https://images.igdb.com/igdb/image/upload/t_th...
                                ...                        
249913    https://images.igdb.com/igdb/image/upload/t_th...
249914    https://images.igdb.com/igdb/image/upload/t_th...
249915    https://images.igdb.com/igdb/image/upload/t_th...
249916    https://images.igdb.com/igdb/image/upload/t_th...
249917    https://images.igdb.com/igdb/image/upload/t_th...
Name: url, Length: 249918, dtype: object

In [16]:
df.to_csv('encodedgames.csv', index=False)